In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from joblib import Parallel, delayed
from itertools import product
import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')
from utilities2014 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

textonmap = dm.load_pipeline_result('texMap', 'npy')
textonmap_viz = dm.load_pipeline_result('texMap', 'png')
n_texton = len(np.unique(textonmap)) - 1

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [3]:
mys, mxs = np.where(dm.mask)
mys = mys.astype(np.int16)
mxs = mxs.astype(np.int16)

n_theta = 12
theta_binwidth = 2*np.pi/n_theta

radius_list = [50,100,200]
n_radius = len(radius_list)
max_radius = radius_list[-1]

box_indices = np.dstack(np.meshgrid(np.arange(-max_radius, max_radius+1), 
                                    np.arange(-max_radius, max_radius+1))).astype(np.int16)
norms = np.linalg.norm(box_indices.astype(np.float), axis=-1)

pie_indices_r = []
circle_indices_r = []

for ri in range(n_radius):
    circle_indices = box_indices[norms <= radius_list[ri]]
    circle_indices_r.append(circle_indices)
    
    angles = np.arctan2(circle_indices[:,0], circle_indices[:,1])
    angles[angles < 0] += 2*np.pi
    angular_bins = np.int0(angles/theta_binwidth)
    pie_indices = [circle_indices[angular_bins==k] for k in range(n_theta)]
    pie_indices_r.append(pie_indices)

In [4]:
pie_mask = [[None for _ in range(n_theta)] for _ in radius_list]
for ri, r in enumerate(radius_list):
    for s in range(n_theta):
        pie_mask[ri][s] = np.zeros((2*r+1,2*r+1), np.bool).copy()
        pie_mask[ri][s][r+pie_indices_r[ri][s][:,0], r+pie_indices_r[ri][s][:,1]] = 1

In [5]:
from scipy.signal import convolve2d, fftconvolve

textonmap_individual_channels = [textonmap == c for c in range(n_texton)]

def compute_pie_histogram(ri, s, pie_mask):
    h = np.zeros((dm.image_height, dm.image_width, n_texton), np.int)
    for c in range(n_texton):
        h[:,:,c] = fftconvolve(textonmap_individual_channels[c], pie_mask, mode='same').astype(np.int)
    dm.save_pipeline_result(h, 'h_r%d_s%d'%(ri,s), 'hdf')

In [6]:
for ri in range(n_radius):
    t = time.time()
    Parallel(n_jobs=16)(delayed(compute_pie_histogram)(ri, s, pie_mask[ri][s]) for s in range(n_theta))
    print time.time() - t

JoblibNameError: JoblibNameError
___________________________________________________________________________
Multiprocessing exception:
    ...........................................................................
/opt/python/lib/python2.7/runpy.py in _run_module_as_main(mod_name='IPython.kernel.__main__', alter_argv=1)
    157     pkg_name = mod_name.rpartition('.')[0]
    158     main_globals = sys.modules["__main__"].__dict__
    159     if alter_argv:
    160         sys.argv[0] = fname
    161     return _run_code(code, main_globals, None,
--> 162                      "__main__", fname, loader, pkg_name)
        fname = '/oasis/projects/nsf/csd181/yuncong/virtualenv-1....ython2.7/site-packages/IPython/kernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'IPython.kernel'
    163 
    164 def run_module(mod_name, init_globals=None,
    165                run_name=None, alter_sys=False):
    166     """Execute a module's code without importing it

..

In [9]:
def chi2s(h1s, h2s):
    return np.nansum((h1s-h2s)**2/(h1s+h2s).astype(np.float), axis=1)

def compute_halfdisc_histogram_diff(H, start_bin, mys, mxs):
    
    n_theta, height, width = H.shape[:3]
    
    Gs = np.zeros((height, width), np.float)

    first_half_bins = np.arange(start_bin, start_bin+n_theta/2)%n_theta
    second_half_bins = np.arange(start_bin+n_theta/2, start_bin+n_theta)%n_theta

    H_halfdisk1 = np.sum(H[first_half_bins[:,None], mys, mxs], axis=0).astype(np.float)
    H_halfdisk2 = np.sum(H[second_half_bins[:,None], mys, mxs], axis=0).astype(np.float)
        
    H_halfdisk1 /= H_halfdisk1.sum(axis=-1)[:,None]
    H_halfdisk2 /= H_halfdisk2.sum(axis=-1)[:,None]
    
    Gs[mys, mxs] = chi2s(H_halfdisk1, H_halfdisk2)

    return Gs

In [ ]:
# %time G = Parallel(n_jobs=16)(delayed(compute_halfdisc_histogram_diff)(s, mys, mxs) for s in range(n_theta/2))

# Gmax = np.zeros((dm.image_height, dm.image_width))
# H = np.zeros((n_theta, dm.image_height, dm.image_width, n_texton), np.int)

# for ri in range(n_radius):
    
#     t = time.time()

#     for s in range(n_theta):
#         with open_file('/home/yuncong/csd395/h_r%d_s%d.hdf'%(ri,s), mode="r") as f:
#             H[s] = f.get_node('/data').read()

#     #     G = np.empty((n_theta/2, height, width))
#     G = [compute_halfdisc_histogram_diff(H, s, mys, mxs) for s in range(n_theta/2)]
#     Gmax = np.maximum(Gmax, np.max(G, axis=0))
    
#     print time.time() - t
    
# Gmax = np.zeros((n_radius, dm.image_height, dm.image_width), np.float)
# for ri in range(n_radius):

In [ ]:
H = np.zeros((n_theta, dm.image_height, dm.image_width, n_texton), np.int)

ri = 0

t = time.time()

for s in range(n_theta):
    with open_file('/home/yuncong/csd395/h_r%d_s%d.hdf'%(ri,s), mode="r") as f:
        H[s] = f.get_node('/data').read()

print time.time() - t

In [10]:
t = time.time()

G = [compute_halfdisc_histogram_diff(H, s, mys, mxs) for s in range(n_theta/2)]
Gmax = np.max(G, axis=0)

print time.time() - t

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:16: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:17: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app


In [11]:
dm.save_pipeline_result(Gmax, 'Gmax', 'npy')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0002/RS141_x5_0002_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_Gmax.npy
